In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from custom_dataset import CustomDataset
from collate import collate_fn
from torch import nn
import pickle
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import librosa

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
#model.to(device)

cuda


In [2]:
def get_pkl(file_path):
    with open(file_path, 'rb') as file:
        loaded_list = pickle.load(file)
    return loaded_list

In [3]:
np_wav = get_pkl('np_wavs_cut.pkl')

In [10]:
np_wav = np_wav[:10000]

In [25]:
del np_wav 

In [13]:
str_list = get_pkl('str_list_cut.pkl')

In [5]:
str_list = str_list[:10000]

In [14]:
print(len(np_wav))
print(len(str_list))

10000
32730


In [12]:
def max_len(np_wav):    
    length = [len(x) for x in np_wav]
    
    combined_list = list(zip(length, np_wav))

    # Sort the combined list based on the elements of the first list
    sorted_list = sorted(combined_list, key=lambda x: x[0])

    # Separate the sorted elements into separate lists
    sorted_first_list, sorted_second_list = zip(*sorted_list)
    
    return sorted_first_list, sorted_second_list

In [13]:
x, y = max_len(np_wav)

In [22]:
for i in y[-10:]:
    mfcc = librosa.feature.mfcc(y=i.astype(float), sr=24000, hop_length=512)
    print(mfcc.shape)

(20, 937)
(20, 937)
(20, 937)
(20, 937)
(20, 937)
(20, 937)
(20, 938)
(20, 938)
(20, 938)
(20, 938)


In [56]:
f, t, processed_data = signal.stft(y[3000], 24000, nperseg=1024)

# Create a new array filled with zeros of the desired shape
new_array = np.pad(processed_data, ((0, 0), (0, 939 - processed_data.shape[1])), mode='constant')

In [7]:
dataset = CustomDataset(np_wav, str_list)
dataloader = DataLoader(dataset, batch_size=256, collate_fn=collate_fn, num_workers=0) # + num thread num_workers=6,

In [ ]:
class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

In [ ]:
class VectorizeChar:
    def __init__(self, max_len=50):
        self.vocab = (
            ["-", "#", "<", ">"]
            + [chr(i + 96) for i in range(1, 27)]
            + [" ", ".", ",", "?"]
        )
        self.max_len = max_len
        self.char_to_idx = {}
        for i, ch in enumerate(self.vocab):
            self.char_to_idx[ch] = i

    def __call__(self, text):
        text = text.lower()
        text = text[: self.max_len - 2]
        text = "<" + text + ">"
        pad_len = self.max_len - len(text)
        return [self.char_to_idx.get(ch, 1) for ch in text] + [0] * pad_len

    def get_vocabulary(self):
        return self.vocab

In [1]:
vocab = (
            ["-", "#", "<", ">"]
            + [chr(i + 96) for i in range(1, 27)]
            + [" ", ".", ",", "?"]
        )

In [3]:
print(vocab)

['-', '#', '<', '>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '?']


In [8]:
for i, data in enumerate(dataloader):
    print(i)
    print(data.shape)

0
torch.Size([256, 20, 938])
1
torch.Size([256, 20, 938])
2
torch.Size([256, 20, 938])
3
torch.Size([256, 20, 938])
4
torch.Size([256, 20, 938])
5
torch.Size([256, 20, 938])
6
torch.Size([256, 20, 938])
7
torch.Size([256, 20, 938])
8
torch.Size([256, 20, 938])
9
torch.Size([256, 20, 938])
10
torch.Size([256, 20, 938])
11
torch.Size([256, 20, 938])
12
torch.Size([256, 20, 938])
13
torch.Size([256, 20, 938])
14
torch.Size([256, 20, 938])
15
torch.Size([256, 20, 938])
16
torch.Size([256, 20, 938])
17
torch.Size([256, 20, 938])
18
torch.Size([256, 20, 938])
19
torch.Size([256, 20, 938])
20
torch.Size([256, 20, 938])
21
torch.Size([256, 20, 938])
22
torch.Size([256, 20, 938])
23
torch.Size([256, 20, 938])
24
torch.Size([256, 20, 938])
25
torch.Size([256, 20, 938])
26
torch.Size([256, 20, 938])
27
torch.Size([256, 20, 938])
28
torch.Size([256, 20, 938])
29
torch.Size([256, 20, 938])
30
torch.Size([256, 20, 938])
31
torch.Size([256, 20, 938])
32
torch.Size([256, 20, 938])
33
torch.Size([256, 

IndexError: list index out of range